# DATA 5310 – Earthquake Risk in Seattle  
## Deep Dive: Data Visualization Notebook

This notebook focuses on **data visualization** for the earthquake risk and seismic retrofit project.  
It assumes the same datasets used in the main analysis:

- `Building_Permits_20251204.csv`
- `CITYPLAN_CRA_*.geojson` (Community Reporting Areas)
- `Unreinforced_Masonry_Buildings_(URM).geojson`
- `Environmentally_Critical_Areas_ECA_*.geojson`
- `OFM_SAEP_BLOCK20_ESTIMATES_SEATTLE_*.geojson`

> **Tip:** Run the notebook top to bottom after updating paths if your files are in a different folder.


In [ ]:
# -----------------------------------------------------------------------------
# 0. Setup
# -----------------------------------------------------------------------------

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import altair as alt
import matplotlib.pyplot as plt

alt.data_transformers.disable_max_rows()

# File paths (edit if needed)
permits_path = "Building_Permits_20251204.csv"
cra_path = "CITYPLAN_CRA_-6672415173103925082.geojson"
urm_path = "Unreinforced_Masonry_Buildings_(URM).geojson"
eca_slide_path = "Environmentally_Critical_Areas_ECA_-8060458720881993457.geojson"
pop_path = "OFM_SAEP_BLOCK20_ESTIMATES_SEATTLE_-7113746441103743061.geojson"

# Load permits
permits = pd.read_csv(permits_path, low_memory=False)
print("Permits shape:", permits.shape)
permits.head(3)

## 1. Identify Seismic Retrofit Permits

We flag permit descriptions that mention seismic or earthquake-related retrofits.


In [ ]:
# -----------------------------------------------------------------------------
# 1.1 Flag seismic retrofit permits
# -----------------------------------------------------------------------------

RETROFIT_PHRASES = [
    "seismic retrofit",
    "seismic upgrade",
    "seismic proof",
    "seismic home retrofit",
    "seismic home upgrade",
    "seismic home proof",
    "seismically retrofit",
    "seismically upgrade",
    "seismically proof",
    "earthquake retrofit",
    "earthquake upgrade",
    "earthquake proof",
    "earthquake home retrofit",
    "earthquake home upgrade",
    "earthquake home proof",
]

def is_retrofit(desc):
    if not isinstance(desc, str):
        return False
    norm = " ".join(desc.split()).lower()
    return any(p in norm for p in RETROFIT_PHRASES)

permits["is_retrofit"] = permits["Description"].apply(is_retrofit)

date_cols = [
    "AppliedDate",
    "IssuedDate",
    "ExpiresDate",
    "CompletedDate",
]
for c in date_cols:
    if c in permits.columns:
        permits[c] = pd.to_datetime(permits[c], errors="coerce")

permits_valid = permits[permits["AppliedDate"].notna()].copy()
permits_valid["year"] = permits_valid["AppliedDate"].dt.year
permits_valid["month"] = permits_valid["AppliedDate"].dt.month
permits_valid["year_month"] = permits_valid["AppliedDate"].dt.to_period("M").dt.to_timestamp()

retrofits = permits_valid[permits_valid["is_retrofit"]].copy()

print("Total permits with AppliedDate:", permits_valid.shape[0])
print("Retrofit permits:", retrofits.shape[0])

## 2. Temporal Patterns of Retrofit Activity

We look at retrofit activity over time at **year** and **month** resolution, and compare to all permits.


In [ ]:
# -----------------------------------------------------------------------------
# 2.1 Yearly counts and retrofit share
# -----------------------------------------------------------------------------

yearly_all = (
    permits_valid.groupby("year")
    .size()
    .reset_index(name="permit_count")
)

yearly_retro = (
    retrofits.groupby("year")
    .size()
    .reset_index(name="retrofit_count")
)

yearly = yearly_all.merge(yearly_retro, on="year", how="left")
yearly["retrofit_count"] = yearly["retrofit_count"].fillna(0)
yearly["retrofit_share"] = yearly["retrofit_count"] / yearly["permit_count"]

yearly

In [ ]:
# Line chart: yearly retrofit count
alt.Chart(yearly).mark_line(point=True).encode(
    x=alt.X("year:O", title="Year"),
    y=alt.Y("retrofit_count:Q", title="Retrofit applications"),
    tooltip=["year", "retrofit_count", "permit_count", alt.Tooltip("retrofit_share:Q", format=".2%")],
).properties(
    width=600,
    height=300,
    title="Yearly Seismic Retrofit Applications",
)

In [ ]:
# Area chart: retrofit share over time
alt.Chart(yearly).mark_area().encode(
    x=alt.X("year:O", title="Year"),
    y=alt.Y("retrofit_share:Q", title="Retrofits as share of all permits", axis=alt.Axis(format="%")),
    tooltip=["year", alt.Tooltip("retrofit_share:Q", format=".2%")],
).properties(
    width=600,
    height=300,
    title="Share of All Permits That Are Seismic Retrofits",
)

In [ ]:
# -----------------------------------------------------------------------------
# 2.2 Monthly dynamics: heatmap by year-month
# -----------------------------------------------------------------------------

monthly_retro = (
    retrofits.groupby("year_month")
    .size()
    .reset_index(name="retrofit_count")
)

monthly_retro["year"] = monthly_retro["year_month"].dt.year.astype(str)
monthly_retro["month"] = monthly_retro["year_month"].dt.month

alt.Chart(monthly_retro).mark_rect().encode(
    x=alt.X("month:O", title="Month"),
    y=alt.Y("year:O", title="Year"),
    color=alt.Color("retrofit_count:Q", title="Retrofit count"),
    tooltip=["year_month:T", "retrofit_count:Q"],
).properties(
    width=500,
    height=400,
    title="Heatmap of Retrofit Applications by Year and Month",
)

## 3. Spatial Context: CRA, Population, and Permits

We now load the spatial layers, join permits to CRAs, and aggregate population to CRA for visualization.


In [ ]:
# -----------------------------------------------------------------------------
# 3.1 Load CRA and population
# -----------------------------------------------------------------------------

cra = gpd.read_file(cra_path)
if "WATER" in cra.columns:
    cra = cra[cra["WATER"] == 0].copy()
cra = cra.to_crs(epsg=4326)

pop = gpd.read_file(pop_path).to_crs(epsg=4326)

# pick a population column
pop_cols = [c for c in pop.columns if c.startswith("POP20")]
if len(pop_cols) == 0:
    raise ValueError("No POP20* columns found in population dataset.")
pop_field = sorted(pop_cols)[-1]  # latest year
print("Using population field:", pop_field)

In [ ]:
# -----------------------------------------------------------------------------
# 3.2 Spatially join permits to CRA
# -----------------------------------------------------------------------------

permits_points = permits_valid.dropna(subset=["Latitude", "Longitude"]).copy()
permits_points_gdf = gpd.GeoDataFrame(
    permits_points,
    geometry=gpd.points_from_xy(permits_points["Longitude"], permits_points["Latitude"]),
    crs="EPSG:4326",
)

permits_cra = gpd.sjoin(
    permits_points_gdf,
    cra[["CRA_NO", "GEN_ALIAS", "geometry"]],
    how="left",
    predicate="within",
)

permits_cra["is_retrofit"] = permits_cra["is_retrofit"].fillna(False)
permits_cra.head(3)

In [ ]:
# -----------------------------------------------------------------------------
# 3.3 Aggregate population by CRA
# -----------------------------------------------------------------------------

pop_cra = gpd.sjoin(
    pop[[pop_field, "geometry"]],
    cra[["CRA_NO", "GEN_ALIAS", "geometry"]],
    how="left",
    predicate="within",
)

cra_pop = (
    pop_cra.groupby(["CRA_NO", "GEN_ALIAS"], as_index=False)[pop_field]
    .sum()
    .rename(columns={pop_field: "population"})
)

cra_pop.head(3)

## 4. CRA-Level Retrofit Metrics

We build CRA-level metrics and visualize them with **choropleths** and **ranked bar charts**.


In [ ]:
# -----------------------------------------------------------------------------
# 4.1 CRA metrics
# -----------------------------------------------------------------------------

cra_permits = (
    permits_cra.groupby(["CRA_NO", "GEN_ALIAS"])
    .agg(
        total_permits=("PermitNum", "count"),
        retrofit_permits=("is_retrofit", "sum"),
    )
    .reset_index()
)

cra_permits = cra_permits.merge(cra_pop, on=["CRA_NO", "GEN_ALIAS"], how="left")

cra_permits["retrofit_share_permits"] = (
    cra_permits["retrofit_permits"] / cra_permits["total_permits"].replace({0: np.nan})
)
cra_permits["retrofit_rate_per_10k"] = (
    cra_permits["retrofit_permits"] / cra_permits["population"].replace({0: np.nan}) * 10000
)

cra_permits.sort_values("retrofit_rate_per_10k", ascending=False).head(10)

In [ ]:
# 4.2 Map: retrofit rate per 10k residents
cra_map_rate = cra.merge(
    cra_permits[["CRA_NO", "retrofit_rate_per_10k"]],
    on="CRA_NO",
    how="left",
)

alt.Chart(cra_map_rate).mark_geoshape().encode(
    color=alt.Color("retrofit_rate_per_10k:Q", title="Retrofits per 10k residents"),
    tooltip=["GEN_ALIAS:N", "retrofit_rate_per_10k:Q"],
).properties(
    width=500,
    height=600,
    title="Retrofit Intensity by CRA (per 10k residents)",
).project("mercator")

In [ ]:
# 4.3 Map: retrofits as share of all permits
cra_map_share = cra.merge(
    cra_permits[["CRA_NO", "retrofit_share_permits"]],
    on="CRA_NO",
    how="left",
)

alt.Chart(cra_map_share).mark_geoshape().encode(
    color=alt.Color("retrofit_share_permits:Q", title="Retrofits / all permits", scale=alt.Scale(scheme="blues")),
    tooltip=["GEN_ALIAS:N", alt.Tooltip("retrofit_share_permits:Q", format=".2%")],
).properties(
    width=500,
    height=600,
    title="Retrofit Share of All Permits by CRA",
).project("mercator")

In [ ]:
# 4.4 Ranked bar chart: top 15 CRAs by retrofit rate per 10k
top15 = cra_permits.sort_values("retrofit_rate_per_10k", ascending=False).head(15)

alt.Chart(top15).mark_bar().encode(
    y=alt.Y("GEN_ALIAS:N", sort="-x", title="CRA"),
    x=alt.X("retrofit_rate_per_10k:Q", title="Retrofits per 10k residents"),
    tooltip=["GEN_ALIAS", "retrofit_rate_per_10k", "retrofit_permits", "population"],
).properties(
    width=600,
    height=400,
    title="Top 15 CRAs by Retrofit Rate per 10k Residents",
)

## 5. Interactive Time–Space Linking

This section builds a **linked view**: a time-series histogram with an interactive brush that filters the map of retrofit permits.


In [ ]:
# -----------------------------------------------------------------------------
# 5.1 Interactive brush: time vs permit locations
# -----------------------------------------------------------------------------

# Retrofit permit points
retrofit_points = retrofits.dropna(subset=["Latitude", "Longitude"]).copy()
retrofit_points_gdf = gpd.GeoDataFrame(
    retrofit_points,
    geometry=gpd.points_from_xy(retrofit_points["Longitude"], retrofit_points["Latitude"]),
    crs="EPSG:4326",
)

# Basemap
cra_alt = alt.Chart(cra).mark_geoshape(
).encode().properties(
    width=450,
    height=550,
)

# Points layer
points_chart = alt.Chart(retrofit_points).mark_circle(size=20, opacity=0.7).encode(
    longitude="Longitude:Q",
    latitude="Latitude:Q",
    tooltip=["PermitNum", "AppliedDate:T", "PermitClassMapped", "EstProjectCost"],
)

# Time histogram with brush
brush = alt.selection_interval(encodings=["x"])

time_hist = alt.Chart(retrofits).mark_bar().encode(
    x=alt.X("yearmonth(AppliedDate):T", title="Application month"),
    y=alt.Y("count()", title="Retrofit count"),
    tooltip=["count()"],
).add_params(brush).properties(
    width=600,
    height=150,
    title="Retrofit Applications Over Time (brush to filter map)",
)

map_view = (cra_alt + points_chart.encode(
    color=alt.condition(brush, alt.value("steelblue"), alt.value("lightgray"))
)).project("mercator").transform_filter(brush).properties(
    title="Retrofit Locations (filtered by time brush)"
)

time_hist & map_view

## 6. URM Buildings: Vulnerability and Hazard Overlays

Here we focus on **Unreinforced Masonry Buildings (URM)**, their vulnerability classes, and their location relative to ECAs.


In [ ]:
# -----------------------------------------------------------------------------
# 6.1 Load URM and compute basic stats
# -----------------------------------------------------------------------------

urm = gpd.read_file(urm_path).to_crs(epsg=4326)
print("URM shape:", urm.shape)
urm[["VULNERABILITY_CLASSIFICATION", "CONFIRMED_RETROFIT"]].head(5)

In [ ]:
urm["VULNERABILITY_CLASSIFICATION"].value_counts(dropna=False)

In [ ]:
# 6.2 Map URMs by vulnerability class
urm_pts = urm.copy()
urm_pts["X"] = urm_pts.geometry.x
urm_pts["Y"] = urm_pts.geometry.y

base = alt.Chart(cra).mark_geoshape().encode().properties(
    width=500,
    height=600,
)

urm_points_chart = alt.Chart(urm_pts).mark_point(filled=True, size=25).encode(
    longitude="X:Q",
    latitude="Y:Q",
    color=alt.Color("VULNERABILITY_CLASSIFICATION:N", title="Vulnerability"),
    tooltip=["MAF_ADDRESS", "VULNERABILITY_CLASSIFICATION", "CONFIRMED_RETROFIT"],
)

(base + urm_points_chart).project("mercator").properties(
    title="URM Buildings by Vulnerability Class",
)

In [ ]:
# 6.3 Faceted view: one map per vulnerability class
alt.Chart(urm_pts).mark_point(filled=True, size=20).encode(
    longitude="X:Q",
    latitude="Y:Q",
    tooltip=["MAF_ADDRESS", "VULNERABILITY_CLASSIFICATION", "CONFIRMED_RETROFIT"],
).properties(
    width=250,
    height=300,
).project("mercator").facet(
    column=alt.Column("VULNERABILITY_CLASSIFICATION:N", title=None)
)

## 7. Hazard Overlays: Slide Areas and URM

We overlay **slide-prone ECAs** with URM locations to visually assess exposure.


In [ ]:
# -----------------------------------------------------------------------------
# 7.1 Load slide ECAs and overlay
# -----------------------------------------------------------------------------

eca_slide = gpd.read_file(eca_slide_path).to_crs(epsg=4326)

slide_chart = alt.Chart(eca_slide).mark_geoshape(opacity=0.4).encode(
    tooltip=[alt.Tooltip("OBJECTID:Q", title="Slide polygon ID")],
)

urm_slide_map = (base + slide_chart + urm_points_chart).project("mercator").properties(
    title="URM Buildings and Slide-Prone ECA Polygons",
)

urm_slide_map

## 8. Risk vs Mitigation Visualization

We build approximate **risk** and **mitigation** indices at CRA level and show them in a scatter plot with quadrant guides.


In [ ]:
# -----------------------------------------------------------------------------
# 8.1 Risk and mitigation indices
# -----------------------------------------------------------------------------

# URM -> CRA
urm_cra = gpd.sjoin(
    urm,
    cra[["CRA_NO", "GEN_ALIAS", "geometry"]],
    how="left",
    predicate="within",
)

vuln_weights = {
    "Medium": 1.0,
    "High": 2.0,
    "Critical": 3.0,
}

urm_cra_risk = (
    urm_cra.assign(weight=lambda df: df["VULNERABILITY_CLASSIFICATION"].map(vuln_weights).fillna(0))
    .groupby(["CRA_NO", "GEN_ALIAS"], as_index=False)
    .agg(
        n_urm=("OBJECTID", "count"),
        risk_weighted=("weight", "sum"),
        n_urm_liq=("ECA_LIQUEFACTION", lambda s: (s == "Yes").sum()),
        n_urm_slide=("ECA_POTENTIAL_SLIDE", lambda s: (s == "Yes").sum()),
        n_urm_retrofit=("CONFIRMED_RETROFIT", lambda s: (s == "Yes").sum()),
    )
)

urm_cra_risk["risk_score"] = (
    urm_cra_risk["risk_weighted"]
    + 0.5 * urm_cra_risk["n_urm_liq"]
    + 0.5 * urm_cra_risk["n_urm_slide"]
)
urm_cra_risk["urm_retrofit_share"] = (
    urm_cra_risk["n_urm_retrofit"] / urm_cra_risk["n_urm"].replace({0: np.nan})
)

cra_risk_mit = cra_permits.merge(
    urm_cra_risk[["CRA_NO", "GEN_ALIAS", "risk_score", "urm_retrofit_share"]],
    on=["CRA_NO", "GEN_ALIAS"],
    how="left",
)

def minmax(s):
    s = s.astype(float)
    return (s - s.min()) / (s.max() - s.min())

cra_risk_mit["risk_index"] = minmax(cra_risk_mit["risk_score"].fillna(0))
cra_risk_mit["mitigation_index"] = minmax(
    cra_risk_mit["retrofit_rate_per_10k"].fillna(0)
    + cra_risk_mit["urm_retrofit_share"].fillna(0)
)

cra_risk_mit.head(5)

In [ ]:
# 8.2 Scatter with quadrant lines
risk_mid = float(cra_risk_mit["risk_index"].median())
mit_mid = float(cra_risk_mit["mitigation_index"].median())

scatter = alt.Chart(cra_risk_mit).mark_circle(size=80, opacity=0.8).encode(
    x=alt.X("risk_index:Q", title="Relative risk"),
    y=alt.Y("mitigation_index:Q", title="Relative mitigation"),
    tooltip=["GEN_ALIAS", "risk_index", "mitigation_index",
             "risk_score", "retrofit_rate_per_10k", "urm_retrofit_share"],
)

hline = alt.Chart(pd.DataFrame({"y": [mit_mid]})).mark_rule().encode(y="y:Q")
vline = alt.Chart(pd.DataFrame({"x": [risk_mid]})).mark_rule().encode(x="x:Q")

(scatter + hline + vline).properties(
    width=500,
    height=400,
    title="Risk vs Mitigation by CRA (quadrants based on median indices)",
)

## 9. Next Steps

This notebook is designed as a **visual companion** to your main analytical notebook and written report.

You can extend it by:
- Adding labels for a few key CRAs in the risk–mitigation scatter.
- Exporting Altair charts as PNG/SVG for your final report slides.
- Creating additional faceted views (e.g., retrofits by permit class over time, or URM maps faceted by ECA flags).